In [1]:
import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

import pandas as pd

In [2]:
# Đọc lại label_mapping, scaler, và label_encoders đã lưu
with open("..\\store\\pickle\\label_mapping.pkl", "rb") as f:
    label_mapping = pickle.load(f)

with open("..\\store\\pickle\\scaler_cat.pkl", "rb") as f:
    scaler_cat = pickle.load(f)

with open("..\\store\\pickle\\scaler_seq.pkl", "rb") as f:
    scaler_seq = pickle.load(f)

with open("..\\store\\pickle\\label_encoders.pkl", "rb") as f:
    label_encoders = pickle.load(f)

cnn_2_inputs_model = tf.keras.models.load_model("..\\store\\models\\cnn_2_inputs_model.keras")

In [3]:
def predict_label(student_data):
    # Tách categorical và sequence features
    categorical_features = ["noisinh", "gioitinh", "khoa", "hedt", "chenhlech_khoahoc", "diem_tt"]
    sequence_features = [f"dtbhk_hk_{i}" for i in range(1, 6)] + \
                        [f"sotchk_hk_{i}" for i in range(1, 6)] + \
                        [f"drl_hk_{i}" for i in range(1, 6)]

    cat_data = np.array([student_data[feature] for feature in categorical_features])
    seq_data = np.array([student_data[feature] for feature in sequence_features]).reshape(1, 5, -1)

    # Encode categorical data
    for i, col in enumerate(categorical_features[:-2]):  # Bỏ 2 cột numeric: chenhlech_khoahoc, diem_tt
        cat_data[i] = label_encoders[col].transform([cat_data[i]])[0]
    
    cat_data = cat_data.astype(float)
    cat_data[-2:] = cat_data[-2:].astype(float)  # Chuyển 2 cột numeric thành float
    
    # Chuẩn hóa categorical data
    cat_data = cat_data.reshape(1, -1)
    cat_data = scaler_cat.transform(cat_data.reshape(1, -1))

    # Chuẩn hóa sequence data
    seq_data = seq_data.reshape(-1, seq_data.shape[-1])  # Reshape để chuẩn hóa
    seq_data = scaler_seq.transform(seq_data)
    seq_data = seq_data.reshape(1, 5, -1)  # Reshape lại cho mô hình

    # Dự đoán
    predicted_label_code = np.argmax(cnn_2_inputs_model.predict([seq_data, cat_data]))
    return label_mapping[predicted_label_code]

# Anvil

In [4]:
import anvil.server

anvil.server.connect("server_MDKXKGHA4SGAJSTPY24O7TGK-3W4M36Q4543DRR2J")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step


E:\Anaconda 3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_layer_5', 'input_layer_6']. Received: the structure of inputs=('*', '*')
  warnings.warn(
Exception in thread Thread-6 (heartbeat_until_reopened):
Traceback (most recent call last):
  File "E:\Anaconda 3\Lib\site-packages\anvil\server.py", line 419, in call
    return _do_call(args, kwargs, fn_name=fn_name)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "E:\Anaconda 3\Lib\site-packages\anvil\server.py", line 409, in _do_call
    return _threaded_server.do_call(args, kwargs, fn_name=fn_name, live_object=live_object)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "E:\Anaconda 3\Lib\site-packages\anvil\_threaded_server.py", line 436, in do_call
    raise error_from_server
anvil._server.AnvilWrappedError: Connection to Anvil Uplink server lost

During handling of the above exce

Anvil websocket closed (code 1006, reason=Going away)
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
Reconnecting Anvil Uplink...
Connecting to wss://anvil.works/uplink
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')
Reconn

In [5]:
@anvil.server.callable
def process_data(noisinh, gioitinh, khoa, hedt, diem_tt, namsinh, khoahoc,
                 dtbhk_hk_1, dtbhk_hk_2, dtbhk_hk_3, dtbhk_hk_4, dtbhk_hk_5,
                 sotchk_hk_1, sotchk_hk_2, sotchk_hk_3, sotchk_hk_4, sotchk_hk_5,
                 drl_hk_1, drl_hk_2, drl_hk_3, drl_hk_4, drl_hk_5):

    # Ép kiểu dữ liệu cho các trường cần là float
    student_data = {
        'noisinh': str(noisinh),
        'gioitinh': str(gioitinh),
        'khoa': str(khoa),
        'hedt': str(hedt),
        
        # Các trường kiểu float
        'diem_tt': float(diem_tt),  
        'dtbhk_hk_1': float(dtbhk_hk_1),
        'dtbhk_hk_2': float(dtbhk_hk_2),
        'dtbhk_hk_3': float(dtbhk_hk_3),
        'dtbhk_hk_4': float(dtbhk_hk_4),
        'dtbhk_hk_5': float(dtbhk_hk_5),
        
        'sotchk_hk_1': float(sotchk_hk_1),
        'sotchk_hk_2': float(sotchk_hk_2),
        'sotchk_hk_3': float(sotchk_hk_3),
        'sotchk_hk_4': float(sotchk_hk_4),
        'sotchk_hk_5': float(sotchk_hk_5),
        
        'drl_hk_1': float(drl_hk_1),
        'drl_hk_2': float(drl_hk_2),
        'drl_hk_3': float(drl_hk_3),
        'drl_hk_4': float(drl_hk_4),
        'drl_hk_5': float(drl_hk_5),
        
        # Các trường kiểu int
        'khoahoc': int(khoahoc),
        'namsinh': int(namsinh),
        
        # Tính toán chenhlech_khoahoc
        'chenhlech_khoahoc': int(khoahoc) - (int(namsinh) - 1987)
    }

    # Dự đoán nhãn
    label = predict_label(student_data)
    
    # Tạo thông điệp phản hồi
    feedback = ""
    if label == "Xuất sắc":
        feedback = "Xin chúc mừng! Tiếp tục giữ vững phong độ này nhé!"
    elif label == "Giỏi":
        feedback = "Đây là thành tích tốt, tuy nhiên bạn có thể cố gắng hơn để đạt Xuất sắc!"
    elif label == "Khá":
        feedback = "Bạn nên tập trung hơn vào các môn học và cải thiện điểm rèn luyện để nâng cao kết quả."
    elif label == "Trung bình khá":
        feedback = "Hãy xem xét lại phương pháp học tập và sắp xếp thời gian hợp lý hơn để cải thiện trong những học kỳ tới."
    
    # Trả về thông báo đầy đủ
    return f"Kết quả của bạn sau 5 học kỳ: {label}. {feedback}"